In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

# print("Path to dataset files:", path)

In [95]:
path = '/Users/ankeshkumar/.cache/kagglehub/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/versions/1/IMDB Dataset.csv'

In [96]:
reviews_df = pd.read_csv(path)

In [97]:
reviews_df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


## Encoding Sentiment Vectors


In [6]:
from sklearn.preprocessing import LabelEncoder

In [92]:
label_encoder = LabelEncoder()
reviews_df['sentiment'] = label_encoder.fit_transform(reviews_df['sentiment'])

In [93]:
reviews_df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [42]:
y = reviews_df['sentiment']
y.head()

0    1
1    1
2    1
3    0
4    1
Name: sentiment, dtype: int64

## Text Cleaning

In [9]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re

ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [10]:
corpus = []
for i in range(0, len(reviews_df)):
    review = re.sub('[^a-zA-Z0-9]', ' ', reviews_df['review'][i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [28]:
corpus[0:2]

['one reviewer mentioned watching 1 oz episode hooked right exactly happened br br first thing struck oz brutality unflinching scene violence set right word go trust show faint hearted timid show pull punch regard drug sex violence hardcore classic use word br br called oz nickname given oswald maximum security state penitentary focus mainly emerald city experimental section prison cell glass front face inwards privacy high agenda em city home many aryan muslim gangsta latino christian italian irish scuffle death stare dodgy dealing shady agreement never far away br br would say main appeal show due fact go show dare forget pretty picture painted mainstream audience forget charm forget romance oz mess around first episode ever saw struck nasty surreal say ready watched developed taste oz got accustomed high level graphic violence violence injustice crooked guard sold nickel inmate kill order get away well mannered middle class inmate turned prison bitch due lack street skill prison exp

In [12]:
corpus[0]

'one reviewer mentioned watching 1 oz episode hooked right exactly happened br br first thing struck oz brutality unflinching scene violence set right word go trust show faint hearted timid show pull punch regard drug sex violence hardcore classic use word br br called oz nickname given oswald maximum security state penitentary focus mainly emerald city experimental section prison cell glass front face inwards privacy high agenda em city home many aryan muslim gangsta latino christian italian irish scuffle death stare dodgy dealing shady agreement never far away br br would say main appeal show due fact go show dare forget pretty picture painted mainstream audience forget charm forget romance oz mess around first episode ever saw struck nasty surreal say ready watched developed taste oz got accustomed high level graphic violence violence injustice crooked guard sold nickel inmate kill order get away well mannered middle class inmate turned prison bitch due lack street skill prison expe

In [13]:
from nltk import sent_tokenize, word_tokenize
words = []
for sent in corpus:
    words.append(word_tokenize(sent.lower()))

In [392]:
# words[:5]

In [366]:
import gensim
import numpy as np

In [367]:
model = gensim.models.Word2Vec(words, window=10, epochs=20, min_count=2)

In [368]:
model.corpus_count

50000

In [369]:
model.epochs

20

In [370]:
model.wv.similar_by_word('horrifying')

[('horrific', 0.7738144397735596),
 ('terrifying', 0.7133768200874329),
 ('frightening', 0.698628842830658),
 ('disturbing', 0.6776716113090515),
 ('shocking', 0.6690909266471863),
 ('chilling', 0.6390683650970459),
 ('unsettling', 0.6346346735954285),
 ('harrowing', 0.6306037902832031),
 ('gruesome', 0.6243882179260254),
 ('nightmarish', 0.5973680019378662)]

In [371]:
# !pip install tqdm

In [372]:
from tqdm import tqdm
def avg_word2vec(doc):
    # Filter only words that exist in the model's vocabulary
    vectors = [model.wv[word] for word in doc if word in model.wv.index_to_key]
    
    # Check if there are vectors to average
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        # Return a zero vector of the same dimension if no words match
        return np.zeros(model.vector_size)

# Create X by applying avg_word2vec to each document in words
X = [avg_word2vec(doc) for doc in tqdm(words)]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [03:06<00:00, 268.32it/s]


In [373]:
X[0]

array([-0.22226457,  0.13322665, -0.08862661, -0.3711489 , -0.12244613,
        0.606003  ,  0.01285876, -0.76574004, -0.01888487,  0.00550692,
       -0.45201918,  1.1800693 , -0.17204162, -0.6546239 ,  0.07476124,
       -0.25856492,  0.21179146,  0.05865158, -0.02761856,  0.8504032 ,
       -0.6608519 ,  1.0788139 , -0.21138614,  0.43277326,  0.31392482,
        0.10082427,  0.4733529 ,  0.5635895 ,  0.62550884,  0.30822772,
        0.06134244,  0.45106742,  0.07271243, -0.1250746 , -0.6153077 ,
       -0.22502328,  0.1086045 ,  0.16759461, -0.12657648,  0.94596237,
        0.29740676,  0.00213285,  0.20468183,  0.21936448, -0.00143678,
        0.04719599,  0.01839209,  0.59448195, -0.44999766, -0.1147849 ,
       -0.360944  ,  0.3116221 ,  0.6113081 , -0.2724321 , -0.6046186 ,
        0.52721363, -0.22018178,  0.5545427 ,  0.47258916, -0.12934776,
        0.15599386, -0.20459248, -0.20523983,  0.16055468,  1.2602025 ,
       -0.7868907 ,  0.1038037 ,  0.1310677 ,  0.71468383, -0.32

In [374]:
X = np.array(X)

## Train test split

In [375]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [376]:
X_train.shape, X_test.shape

((33500, 100), (16500, 100))

In [377]:
y_train.shape, y_test.shape

((33500,), (16500,))

## Model Training

In [378]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# sentiment_detection_model = RandomForestClassifier().fit(X_train, y_train)

sentiment_detection_model = SVC(kernel='rbf', C=1.0, gamma='scale')
sentiment_detection_model.fit(X_train, y_train)

SVC()

In [379]:
y_pred = sentiment_detection_model.predict(X_test)

In [380]:
y_pred[:50]

array([0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 0])

In [381]:
np.array(y_test[:50])

array([1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 1])

In [386]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [387]:
score = accuracy_score(y_test, y_pred)

In [388]:
score

0.8898787878787878

In [389]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.88      0.89      8208
           1       0.88      0.90      0.89      8292

    accuracy                           0.89     16500
   macro avg       0.89      0.89      0.89     16500
weighted avg       0.89      0.89      0.89     16500



In [390]:
c_matrix = confusion_matrix(y_test, y_pred)

In [391]:
c_matrix

array([[7215,  993],
       [ 824, 7468]])